# Artist Similarity with Graph Neural Network 2nd Notebook

In this notebook are shown the performances of the networks obtained from the training as described in the first notebook.  
In addition to the authors we have seen the quality of the recommended artists, with a query artists and the aid of the K-NN computation.  
It is in our interest to compare all the architectures and to see how the GAT ourtperforms in the results the GraphSAGE layer.

* Another important aspect that we see in this notebook is the possibility to create non-existing artists by only specifiyng fake relationship in the Graph with existing artists. The procedure is simple, we just create a feature vector for the fake artist and then we embed it with the other samples. This procedure also require that are specified one or more existing artist that are related to the fake one, in this way is possible to mix musical genres and see what are the recommended artists.

In [3]:
# !apt install cm-super
# # ! sudo apt-get install texlive-latex-recommended 
# # ! sudo apt-get install dvipng texlive-latex-extra texlive-fonts-recommended  
# ! wget http://mirrors.ctan.org/macros/latex/contrib/type1cm.zip 
# ! unzip type1cm.zip -d /tmp/type1cm
# ! cd /tmp/type1cm/type1cm/ && sudo latex type1cm.ins
# ! sudo mkdir /usr/share/texmf/tex/latex/type1cm 
# ! sudo cp /tmp/type1cm/type1cm/type1cm.sty /usr/share/texmf/tex/latex/type1cm 
# ! sudo texhash 
!pip install matplotlib==3.2.2


DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
import numpy as np
# import plotly.express as px
# import plotly.graph_objects as go
import json
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics.functional import pairwise_euclidean_distance
from torch_geometric.nn import GATConv, SAGEConv
from torch.optim import lr_scheduler
import random
from random import choice,randrange
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import math
import time
from torch_geometric import seed_everything

random_seed=280085

seed_everything(random_seed)

2.2.1+cu121


/home/peppe/anaconda3/envs/GNRR2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/peppe/anaconda3/envs/GNRR2/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
from architectures import *
from utils import *

2.2.1+cu121


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

With the help of the [Torch geometric framework](https://pytorch-geometric.readthedocs.io/en/latest/) was really easy to handle the graph attributes and nodes and then the training of the GNNs.

In [7]:
X = torch.load('instance').T.to(device)      # Instance matrix
A = torch.load('adjacencyCOO').to(device)    # Adjacency matrix in the COO format, that is that supported by torch geometric
A1 = torch.load('adjacency').to(device)      # Normal adjacency matrix format is obtained with torch.load('adjacency')
num_samples = X.shape[0]
print(num_samples)

11261


In [8]:
filt = True


art_of_interest = torch.load('intrst_artists.pt')
labels = torch.load('labels.pt')
if filt:
  X = X[art_of_interest].detach()
  A1 = A1[art_of_interest, :][:, art_of_interest].detach()
  labels = labels[art_of_interest].detach()
  A = torch.nonzero(A1).T.type(torch.LongTensor).detach()

In [9]:
''' These variables contain the information about the artists' names, and their position in the dataset, this makes easy to look for their name and to better draw conclusions at inference time '''
num2artist = load_data('data/artist_genres.json')

num2artist = {key: num2artist[key][1] for key in num2artist}
artist2num = {num2artist[key]:key for key in num2artist}

##### Data in the dataset corrensponds to a set of artist, we have stored their names in two dictionaries. These two data structure are fundamental to keep track of their names and eventually to do more specific experiments.

## Import the data with Torch geometric:


In [10]:
''' In order to conduct the experiments was fundamental to split the dataset, either the nodes and also the edges.
    The splitting was performed according to the information in the paper, and considering the fact that a lot of date were lost in the preprocessing part.'''
device = 'cuda' if torch.cuda.is_available() else 'cpu'

from torch_geometric.data import Data
from torch_geometric.utils import structured_negative_sampling
data = Data(x=X, edge_index = A, y = labels).to(device)


In [11]:
# Number of layers #
# G1 = GraphSage(1)
# G2 = GraphSage(2)
# G3 = GraphSage(3)
n_heads = 1
n_layers = 2
GATm = GATSYFC(n_heads, n_layers)
load_model('./models/GATSY.pt', GATm, device)


RuntimeError: Error(s) in loading state_dict for GATSY:
	Missing key(s) in state_dict: "GAT1.lin.weight", "GAT_l.0.lin.weight". 
	Unexpected key(s) in state_dict: "GAT1.lin_src.weight", "GAT1.lin_dst.weight", "GAT_l.0.lin_src.weight", "GAT_l.0.lin_dst.weight". 

## What happens if we create an unreal artist?
* Since we are able now to embed artists, it is easy to augment our Graph Data Structure with new information, and we would expect to obtain plausible results as well.
* Thus we insert a new artist by specifying its connections in the graph, and then we compute its features as a linear combination of its neighbors features, namely the average.

* In the next cell we can either look for the neighbors of existing artist and also to create new artist and their neighborhoods.

In [11]:
def get_nearest_artists(embedding, artist_name, K, artist_to_id, id_to_artist):
  '''To get the nearest artist we use the K-NN algorithm:
      - embedding:   Is the embedding of artist,
      - artist_name: Query artist for which we are looking at its neighbors,
      - K:           Number of neighbors,
      - artist_to_id, and id_to_artist are the dictionary that keep track of the artists and their ids. '''
  Knew = K+50
  T=embedding.detach().to(torch.device("cpu")).numpy()
  neigh=NearestNeighbors(n_neighbors=Knew,algorithm='kd_tree').fit(T)#With the K-NN we get the nearest 
  if not filt:
    dist,ind = neigh.kneighbors(T[int(artist_to_id[artist_name])].reshape((1,-1)))
  else:
    idx = torch.nonzero(art_of_interest.where(art_of_interest == int(artist_to_id[artist_name]), torch.tensor([0]))).item()
    dist,ind = neigh.kneighbors(T[idx].reshape((1,-1)))


  neighbors_list = list(ind[0])[1:]
  dist_list = list(dist[0])[1:]
  neighbors_ = []
  c = 1
  while len(neighbors_)<K:
    if id_to_artist[str(neighbors_list[c])]!=None and id_to_artist[str(neighbors_list[c])] not in friend_artist_list:
      if not filt:
        neighbors_.append((id_to_artist[str(neighbors_list[c])],round(dist_list[c],4)))
        
      else:
        neighbors_.append((id_to_artist[str(art_of_interest[neighbors_list[c]].item())],round(dist_list[c],4)))


      c+=1
    else:
      c+=1

  #neighbors_list = [id_to_artist[str(artist)] for artist in neighbors_list if str(artist) in id_to_artist]
  
  return neighbors_

def get_embeddings(model, data):
  ''' This function simply computes the embeddings given a model name, and the data that we are using '''
  
  embedding = model(data.x, data.edge_index.to(device))

  return embedding

def add_new_artist(artist_query, friend_artist_list): 
  ''' This function augment the dataset if the artist_query is not already present in the dataset. 
      - artist_query:       artist for which we conduct the search,
      - friend_artist_list: list of correlated artists.                                            '''


  X_new = X.clone()
  A_new = A.clone()
  artist2num_new = artist2num.copy()
  num2artist_new = num2artist.copy()
  if artist_query not in artist2num and len(friend_artist_list) != 0:
    print("{} does not exist in  the dataset, or in real life. \n But we still can create it!".format(artist_query))
    artist2num_new[artist_query] = str(X_new.shape[0])
    num2artist_new[str(X_new.shape[0])] = artist_query
    feat_sum = torch.zeros(2613, device = device)
    for artist in friend_artist_list:
      if artist not in artist2num_new:
        print("{} is not in the dataset, so it is not valid for the neighbors list".format(artist))
      artist_num = artist2num_new[artist]
      A_new = torch.cat((A_new, torch.tensor([[int(artist2num_new[artist_query])],[int(artist2num_new[artist])]], device = device)), dim = 1)
      A_new = torch.cat((A_new, torch.tensor([[int(artist2num_new[artist])],[int(artist2num_new[artist_query])]], device = device)), dim = 1)
      feat_sum += X_new[int(artist2num_new[artist])]
    feat_sum /=len(friend_artist_list)
    X_new = torch.cat((X_new, feat_sum.unsqueeze(0)), dim = 0)

    data = Data(x=X_new, edge_index = A_new)
    print("\n{} has been created considering its neighbors:\n {}\n".format(artist_query, friend_artist_list))
  else:
    print("{} is an existing artist".format(artist_query))
    data = Data(x=X, edge_index = A)


  return data, artist2num_new, num2artist_new


data_n = data
artist2num_new = artist2num
num2artist_new = num2artist




Please run the 'friend_artist_list' before. Obviously it is required to specify the desired artist to correlate someone with the fictitious artist.

In [ ]:
friend_artist_list = ['Eminem']
embedding = get_embeddings(model = GATm.to(device), data = data_n)
artist_name = 'Ludwig van Beethoven'

if artist_name in list(artist2num_new.keys()):
    print(get_nearest_artists(embedding, artist_name, K = 10, artist_to_id = artist2num_new, id_to_artist = num2artist_new))
else:
    data_n, artist2num_new, num2artist_new = add_new_artist(artist_name, friend_artist_list = friend_artist_list)
    print('Run again the cell!!!')


In [ ]:
label_diz = load_data('data/encode_labels.json') # Genre to idx
label_diz2 = {label_diz[key] : key for key in label_diz} # Idx to genre
label_diz2

In [ ]:
label_diz

In [59]:
lista = [label_diz[str(l.item())] for l in labels]

In [ ]:
occurrencies = json.load(open('data/occurrencies_real.json'))

def get_top_k(dictionary, lista, k):
    sorted_dict = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    top_k = sorted_dict[:k]
    print(top_k)
    top_k = [key for key, value in top_k]
    lista = [l for l in lista if l in top_k]

    return top_k, lista

top_k_list, lista_n = get_top_k(occurrencies, lista, k=10)
boolean_list = torch.tensor([True if l in top_k_list else False for l in lista])

In [ ]:
import seaborn as sns
import pandas as pd
from pylab import rcParams
from matplotlib import gridspec
import matplotlib
import matplotlib.font_manager
custom = sns.color_palette("Paired", 25)

import matplotlib.pyplot as plt
def plot_cluster(embedding1, embedding2, labels, n_clusters):
  DataF1 = pd.DataFrame()
  DataF1['x'] = embedding1[:,0]
  DataF1['y'] = embedding1[:,1]
  DataF1['labels'] = labels


  DataF2 = pd.DataFrame()
  DataF2['x'] = embedding2[:,0]
  DataF2['y'] = embedding2[:,1]
  DataF2['labels'] = labels

#   DataF3 = pd.DataFrame()
#   DataF3['x'] = embedding3[:,0]
#   DataF3['y'] = embedding3[:,1]
#   DataF3['labels'] = clust_obj1.predict(embedding_emb)

#   DataF4 = pd.DataFrame()
#   DataF4['x'] = embedding4[:,0]
#   DataF4['y'] = embedding4[:,1]
#   DataF4['labels'] = clust_obj2.predict(random_embedding_emb)
  
  fig1, axes1 = plt.subplots(1, 2, figsize=(20, 7))

  # fig1.suptitle('Representantion of the two types of input features, after the dimensionality reduction')
  sns.scatterplot(ax = axes1[0],
      x="x", y="y",
      data=DataF1,
      hue="labels",
      palette=custom,
      legend = 'full',
      alpha=0.3
  )

  axes1[0].set_title('Low level features')

  sns.scatterplot(ax = axes1[1],
      x="x", y="y",
      data=DataF2,
      hue="labels",
      palette=custom,
      legend = False,
      alpha=0.3
  )
  axes1[1].set_title('Embedding')
  # axes1[0].legend(title='', loc='upper left', labels=[label_diz[str(genre_idx)] for genre_idx in range(len(label_diz2))])

  plt.subplots_adjust(left=0.1,
                  bottom=0.1,
                  right=0.9,
                  top=0.9,
                  wspace=0.3,
                  hspace=0.4)
  
  plt.savefig('comparisons.pdf')


  plt.show()
#   fig2, axes2 = plt.subplots(1, 2, figsize=(20, 7))
#   # fig2.suptitle('Representantion of the embeddings, after the dimensionality reduction')

#   sns.scatterplot(ax = axes2[0],
#       x="x", y="y",
#       data=DataF3,
#       hue="labels",
#       palette=sns.color_palette("hls", K),
#       legend=False,
#       alpha=0.3
#   )
#   axes2[0].set_title('Low level features')
#   sns.scatterplot(ax = axes2[1],
#       x="x", y="y",
#       data=DataF4,
#       hue="labels",
#       palette=sns.color_palette("hls", K),
#       legend=False,
#       alpha=0.3
#   )
#   axes2[1].set_title('Random features')
#   plt.subplots_adjust(left=0.1,
#                     bottom=0.1,
#                     right=0.9,
#                     top=0.9,
#                     wspace=0.4,
#                     hspace=0.4)
#   plt.savefig('dimred2.pdf')
#   plt.show()

  
params = {
   'axes.labelsize': 26,
   'font.family': 'Times New Roman',
   'font.size': 25,
   'legend.fontsize': 10,
   'xtick.labelsize': 13,
   'ytick.labelsize': 13,
   'text.usetex': False,
   'figure.figsize': [100*1.5, 95*1.5]
   }


rcParams.update(params)
matplotlib.rc('pdf', fonttype=42)
matplotlib.font_manager._rebuild()

instance_emb = torch.load('embeds/low_level.pt')
embedding_emb = torch.load('embeds/embedding_lab.pt')


# plot_cluster(new_emb, clustering, K, 'K-means clusters')
plot_cluster(np.array(instance_emb), np.array(embedding_emb), labels = lista, n_clusters = 25)
# plot_cluster(new_emb, gmm, K, 'Gaussian-Mixture clusters')


In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
from pylab import rcParams
import matplotlib.pyplot as plt
import matplotlib

custom = sns.color_palette("Paired", 25)

def plot_cluster(embedding1, embedding2, labels, n_clusters):
    DataF1 = pd.DataFrame()
    DataF1['x'] = embedding1[:,0][boolean_list]
    DataF1['y'] = embedding1[:,1][boolean_list]
    print(len(DataF1['x']))
    
    DataF1['labels'] = labels

    DataF2 = pd.DataFrame()
    DataF2['x'] = embedding2[:,0][boolean_list]
    DataF2['y'] = embedding2[:,1][boolean_list]
    DataF2['labels'] = labels

    fig1, axes1 = plt.subplots(1, 2, figsize=(20, 7))

    # Plot first scatterplot
    sns.scatterplot(ax=axes1[0],
        x="x", y="y",
        data=DataF1,
        hue="labels",
        palette=custom,
        alpha=0.3,
        legend=False  # Disable the legend here
    )

    # Plot second scatterplot
    sns.scatterplot(ax=axes1[1],
        x="x", y="y",
        data=DataF2,
        hue="labels",
        palette=custom,
        alpha=0.3,
        legend=False  # Disable the legend here
    )

    # Generate custom legend
    unique_labels = np.unique(labels)
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=custom[i], markersize=10) for i in range(len(unique_labels))]
    labels = [f'{i}' for i in unique_labels]
    
    fig1.legend(handles, labels, loc='center left', bbox_to_anchor=(0.38, 0.85), title='Labels')

    # Adjust subplot parameters
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.85, top=0.9, wspace=0.3, hspace=0.4)

    plt.savefig('comparisons.pdf')
    plt.show()

params = {
    'axes.labelsize': 26,
    'font.family': 'Times New Roman',
    'font.size': 25,
    'legend.fontsize': 10,
    'xtick.labelsize': 13,
    'ytick.labelsize': 13,
    'text.usetex': False,
    'figure.figsize': [100*1.5, 95*1.5]
}

rcParams.update(params)
matplotlib.rc('pdf', fonttype=42)
matplotlib.font_manager._rebuild()

# Replace 'lista' with your actual labels list
plot_cluster(np.array(instance_emb), np.array(embedding_emb), labels=lista_n, n_clusters=25)


In [13]:
import random
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt

def convert_to_networkx(graph, n_sample=None):

    g = to_networkx(graph.cpu(), node_attrs=["x"])
    y = graph.y.cpu().numpy()

    if n_sample is not None:
        sampled_nodes = random.sample(g.nodes, n_sample)
        g = g.subgraph(sampled_nodes)
        y = y[sampled_nodes]

    return g, y


def plot_graph(g, y):

    plt.figure(figsize=(9, 7))
    nx.draw_spring(g, node_size=30, arrows=False, node_color=y)
    plt.show() 

In [ ]:
g, y = convert_to_networkx(data)
plot_graph(g, y)


In [ ]:
len(lista)